In [1]:
import numpy as np
import FileManager
import scipy.sparse as sp

from Recommenders.ScoresHybridRecommender import ScoresHybridRecommender
from Recommenders.ScoresHybrid2Recommender import ScoresHybrid2Recommender
from Recommenders.ScoresHybrid3Recommender import ScoresHybrid3Recommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender

In [2]:
#ratings = FileManager.load_data()
#users_to_recommend = np.array(FileManager.load_target()).squeeze()
#urm_all,urm_train,urm_validation= FileManager.split_data(ratings)
#urm_train_validation = urm_train + urm_validation

> Importing file...
> Importing file... Completed!
> Importing file...
> Importing file... Completed!


In [2]:
urm_train=sp.load_npz("saved_urm/urm_train.npz")
urm_validation=sp.load_npz("saved_urm/urm_validation.npz")
urm_train_validation=sp.load_npz("saved_urm/urm_train_validation.npz")

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13646 ( 0.0%) Users that have less than 1 test interactions


SLIM

In [ ]:
################ TRAIN SLIM ################
slimElasticNet= SLIMElasticNetRecommender(urm_train)
slimElasticNet.fit(topK=707, l1_ratio=3.7848901259206446e-05, alpha=0.8041273931917446)

In [ ]:
################ SAVE SLIM ################
slimElasticNet.save_model(folder_path='saved_models/train/',file_name='slimElasticNet')

In [4]:
################ LOAD SLIM ################
slimElasticNetLoaded= SLIMElasticNetRecommender(urm_train)
slimElasticNetLoaded.load_model(folder_path='saved_models/train/',file_name='SLIMElasticNet_best_latest')

SLIMElasticNetRecommender: Loading model from file 'saved_models/train/SLIMElasticNet_best_latest'
SLIMElasticNetRecommender: Loading complete


PURE SVD

In [27]:
################ TRAIN PURE SVD ################
pureSVD= PureSVDRecommender(urm_train)
pureSVD.fit(num_factors=32)

PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 3.55 sec


In [10]:
################ SAVE  PURE SVD ################
pureSVD.save_model(folder_path='saved_models/train/',file_name='pureSVD')

PureSVDRecommender: Saving model in file 'saved_models/train/pureSVD'
PureSVDRecommender: Saving complete


In [8]:
################ LOAD PURE SVD ################
pureSVDloaded= PureSVDRecommender(urm_train)
pureSVDloaded.load_model(folder_path='saved_models/train/',file_name='pureSVD')

PureSVDRecommender: Loading model from file 'saved_models/train/pureSVD'
PureSVDRecommender: Loading complete


USER KNN

In [11]:
################ TRAIN USER KNN ################
userKNN=UserKNNCFRecommender(urm_train)
userKNN.fit(topK=677, shrink=302, similarity='cosine', normalize=True, feature_weighting='TF-IDF')


Similarity column 13650 (100.0%), 371.49 column/sec. Elapsed time 36.74 sec


In [12]:
################ SAVE  USER KNN ################
userKNN.save_model(folder_path='saved_models/train/',file_name='userKNNcf')

UserKNNCFRecommender: Saving model in file 'saved_models/train/userKNNcf'
UserKNNCFRecommender: Saving complete


In [6]:
################ LOAD USER KNN ################
userKNNLoaded= UserKNNCFRecommender(urm_train)
userKNNLoaded.load_model(folder_path='saved_models/train/',file_name='UserKNNCF_best')

UserKNNCFRecommender: Loading model from file 'saved_models/train/UserKNNCF_best'
UserKNNCFRecommender: Loading complete


HYBRID

In [25]:
scoreshybridrecommender= ScoresHybridRecommender(urm_train,slimElasticNetLoaded,userKNNLoaded)
#scoreshybridrecommender.fit(alpha=0.77535)
#scoreshybridrecommender.fit(alpha=0.8109400848162426)
scoreshybridrecommender.fit(alpha=0.99)

In [28]:
scoreshybrid2recommender= ScoresHybridRecommender(urm_train,slimElasticNetLoaded,pureSVD)
scoreshybrid2recommender.fit(alpha=0.77535)

In [9]:
from Recommenders.BaseRecommender import BaseRecommender
import scipy.sparse as sps


class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2

    def fit(self, alpha=0.5):
        self.alpha = alpha

    def _compute_item_score(self, user_id_array, items_to_compute):

        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array,items_to_compute)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array,items_to_compute)

        item_weights = item_weights_1 * self.alpha + item_weights_2 * (1 - self.alpha)

        return item_weights


In [29]:

result_df, _ = evaluator_validation.evaluateRecommender(scoreshybrid2recommender)
print(result_df['MAP'])

EvaluatorHoldout: Processed 13646 (100.0%) in 47.83 sec. Users per second: 285
cutoff
10    0.245009
Name: MAP, dtype: object


In [26]:
for rec in [slimElasticNetLoaded,scoreshybridrecommender]:
    result_df, _ = evaluator_validation.evaluateRecommender(rec)
    print(result_df['MAP'])

EvaluatorHoldout: Processed 13646 (100.0%) in 53.41 sec. Users per second: 255
cutoff
10    0.243595
Name: MAP, dtype: object
EvaluatorHoldout: Processed 13646 (100.0%) in 1.15 min. Users per second: 198
cutoff
10    0.244281
Name: MAP, dtype: object


In [23]:
from skopt.space import Real, Integer, Categorical
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Evaluation.Evaluator import EvaluatorHoldout
import os


hyperparameters_range_dictionary = {
    "alpha": Real(low = 0.01, high = 1, prior = 'log-uniform'),
}


output_folder_path = "Optimization_FINAL_HYBRID/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 60  # NUMBER OF CASES
n_random_starts = int(n_cases*0.15)
metric_to_optimize = "MAP"
cutoff_to_optimize = 10


recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train,slimElasticNetLoaded,userKNNLoaded],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS =  {}    # Additiona hyperparameters for the fit function
)
#recommender_input_args_last_test = SearchInputRecommenderArgs( CONSTRUCTOR_POSITIONAL_ARGS = [urm_train_validation,slimElasticNetLoaded,pureSVDloaded,userKNNLoaded],     # For a CBF model simply put [URM_train_validation, ICM_train] CONSTRUCTOR_KEYWORD_ARGS = {}, FIT_POSITIONAL_ARGS = [], FIT_KEYWORD_ARGS = {}   # Additiona hyperparameters for the fit function)

recommender_class = ScoresHybridRecommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation)



In [24]:
# RUN THE OPTIMIZER

hyperparameterSearch.search(recommender_input_args,
                       #recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "no",
                       resume_from_saved=True,
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

SearchBayesianSkopt: Extending previous number of cases from 60 to 65.

SearchBayesianSkopt: Resuming 'ScoresHybridRecommender'... Loaded 60 configurations.
Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Search interrupted due to ValueError. The evaluated configurations may have had all the same value.

